In [2]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()

import qubx
%qubxd

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.impl.ccxt_connector import CCXT_connector

 >  [dev] installed cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# Listen to data

In [9]:
from ccxt.base.exchange import Exchange
from threading import Thread

async def test(exchange: Exchange, symbols):
    method = 'watchTrades'
    if exchange.has[method]:
        while True:
            try:
                trades = await exchange.watch_trades_for_symbols(symbols)
                num_trades = len(trades)
                trade = trades[-1]
                print(
                    exchange.iso8601(exchange.milliseconds()), 
                    trade['symbol'], trade['datetime'], trade['price'], trade['amount'], 'received', num_trades
                )
            except Exception as e:
                await exchange.close()
                raise e
    else:
        raise Exception(exchange.id + ' ' + method + ' is not supported or not implemented yet')


In [4]:
bu = cxp.binanceusdm()

In [10]:
class RunThread(Thread):
    def __init__(self, func, args, kwargs):
        self.func = func
        self.args = args
        self.kwargs = kwargs
        self.result = None
        super().__init__()

    def run(self):
        self.result = asyncio.run(self.func(*self.args, **self.kwargs))

In [12]:
thread = RunThread(test, (bu, ['BTCUSDT', 'ETHUSDT']), {})
thread.start()

2024-03-15T16:07:47.970Z BTC/USDT:USDT 2024-03-15T16:07:48.027Z 67725.9 0.002 received 1
2024-03-15T16:07:48.017Z ETH/USDT:USDT 2024-03-15T16:07:48.062Z 3672.34 0.007 received 4
2024-03-15T16:07:48.017Z ETH/USDT:USDT 2024-03-15T16:07:48.062Z 3672.34 0.032 received 1
2024-03-15T16:07:48.073Z BTC/USDT:USDT 2024-03-15T16:07:48.123Z 67725.9 0.002 received 2
2024-03-15T16:07:48.081Z ETH/USDT:USDT 2024-03-15T16:07:48.144Z 3672.34 0.203 received 2
2024-03-15T16:07:48.090Z BTC/USDT:USDT 2024-03-15T16:07:48.153Z 67726.5 0.009 received 2
2024-03-15T16:07:48.139Z BTC/USDT:USDT 2024-03-15T16:07:48.167Z 67726.6 0.017 received 1
2024-03-15T16:07:48.246Z BTC/USDT:USDT 2024-03-15T16:07:48.279Z 67726.6 0.002 received 4
2024-03-15T16:07:48.247Z BTC/USDT:USDT 2024-03-15T16:07:48.280Z 67726.6 0.018 received 5
2024-03-15T16:07:48.247Z BTC/USDT:USDT 2024-03-15T16:07:48.281Z 67726.6 0.002 received 5
2024-03-15T16:07:48.248Z BTC/USDT:USDT 2024-03-15T16:07:48.281Z 67726.6 0.004 received 2
2024-03-15T16:07:48.3

In [ ]:
thread.join()

In [ ]:
asyncio.run(test(bu, ['BTCUSDT', 'ETHUSDT']))